In [1]:
import json
from pymongo import MongoClient
import random 
from datetime import datetime
from bson import ObjectId
import requests
import pandas as pd

In [2]:
dbName = "dbChat"
mongodbURL = f"mongodb://localhost/{dbName}"
print(mongodbURL)
client = MongoClient(mongodbURL, connectTimeoutMS=2000,serverSelectionTimeoutMS=2000)
db = client.get_database()
#cur = db.list_collections()


mongodb://localhost/dbChat


In [3]:
path='../quotes-100-en.json'
with open(path,'r') as file:
    data=json.load(file)



In [4]:
users=list(data.keys())
messages=list(data.values())
chat_ids=[1,2,3,4,5,6,7,8,9]

In [ ]:
'''#build the list of dictionaries for each message
list_of_messages=[]
for i,message in enumerate(messages):
    now = datetime.now()
    dt_string = now.strftime("%d/%m/%Y %H:%M:%S")
    dic={'user_id':random.choice(users),
         'text':message,
         'time':dt_string,
         'chat_id':[],
    
     }
    list_of_messages.append(dic)
#messages_ids=db.messages.insert_many(list_of_messages)'''

In [8]:
#build the list of dictionaries for users
list_of_users=[]
for i,user in enumerate(users):
    now = datetime.now()
    dt_string = now.strftime("%d/%m/%Y %H:%M:%S")
    dic={'user_name':random.choice(users),
         'insertion_date':dt_string,
         'chats_list':[]
    
         }
    list_of_users.append(dic)
users_ids=db.users.insert_many(list_of_users)

In [9]:
#build the list of dictionaries for chats
list_of_chats=[]
for chat in chat_ids:
    now = datetime.now()
    dt_string = now.strftime("%d/%m/%Y %H:%M:%S")
    dic={'chat_name':random.choice(chat_ids),
         'creation_date':dt_string,
         'users_list':[random.choice(list(users_ids.inserted_ids)) for i in range(50)],
         'messages_list':[]
    
     }
    list_of_chats.append(dic)
    
chats_ids=db.chats.insert_many(list_of_chats)

In [11]:
##populate the database with the messages with the api

messages_ids=[]
for chat_id in chats_ids.inserted_ids:
    for i in range(90):#add 10 messages
        chat_id=str(chat_id)
        user=random.choice(list(users_ids.inserted_ids))
        #add the user to the chat
        r = requests.get(f'http://localhost:3500/chat/{chat_id}/adduser?user_id={user}')
       
        text=random.choice(messages)
        r = requests.get(f'http://localhost:3500/chat/{chat_id}/addmessage?user_id={user}&text={text}')
        
        messages_ids.append(r)
        

In [12]:
chats_ids.inserted_ids

[ObjectId('5ead2b381f313a96eed3c7e5'),
 ObjectId('5ead2b381f313a96eed3c7e6'),
 ObjectId('5ead2b381f313a96eed3c7e7'),
 ObjectId('5ead2b381f313a96eed3c7e8'),
 ObjectId('5ead2b381f313a96eed3c7e9'),
 ObjectId('5ead2b381f313a96eed3c7ea'),
 ObjectId('5ead2b381f313a96eed3c7eb'),
 ObjectId('5ead2b381f313a96eed3c7ec'),
 ObjectId('5ead2b381f313a96eed3c7ed')]

In [13]:
users_ids.inserted_ids


[ObjectId('5ead2b331f313a96eed3ab3d'),
 ObjectId('5ead2b331f313a96eed3ab3e'),
 ObjectId('5ead2b331f313a96eed3ab3f'),
 ObjectId('5ead2b331f313a96eed3ab40'),
 ObjectId('5ead2b331f313a96eed3ab41'),
 ObjectId('5ead2b331f313a96eed3ab42'),
 ObjectId('5ead2b331f313a96eed3ab43'),
 ObjectId('5ead2b331f313a96eed3ab44'),
 ObjectId('5ead2b331f313a96eed3ab45'),
 ObjectId('5ead2b331f313a96eed3ab46'),
 ObjectId('5ead2b331f313a96eed3ab47'),
 ObjectId('5ead2b331f313a96eed3ab48'),
 ObjectId('5ead2b331f313a96eed3ab49'),
 ObjectId('5ead2b331f313a96eed3ab4a'),
 ObjectId('5ead2b331f313a96eed3ab4b'),
 ObjectId('5ead2b331f313a96eed3ab4c'),
 ObjectId('5ead2b331f313a96eed3ab4d'),
 ObjectId('5ead2b331f313a96eed3ab4e'),
 ObjectId('5ead2b331f313a96eed3ab4f'),
 ObjectId('5ead2b331f313a96eed3ab50'),
 ObjectId('5ead2b331f313a96eed3ab51'),
 ObjectId('5ead2b331f313a96eed3ab52'),
 ObjectId('5ead2b331f313a96eed3ab53'),
 ObjectId('5ead2b331f313a96eed3ab54'),
 ObjectId('5ead2b331f313a96eed3ab55'),
 ObjectId('5ead2b331f313a

# Nltk analysis

In [ ]:
!pip3 install nltk

In [5]:
import nltk
from nltk.sentiment.vader import SentimentIntensityAnalyzer
nltk.download("vader_lexicon")

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     /Users/silviaserafini/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


True

In [6]:
sia = SentimentIntensityAnalyzer()

In [15]:
import ast
chat_id='5ead2b381f313a96eed3c7e7'
r = requests.get(f'http://localhost:3500//chat/{chat_id}/list')
messages1=r.json()
#print(messages1)

users_messages={}
#prendo tutti i messaggi di un utente e li unisco
for k,v in messages1.items():
    mes_user=db.messages.find_one({'_id':ObjectId(k)})#fetcg=h the message
    user=mes_user['user_id']#fetch the user
    user_texts=list(db.messages.find({"user_id":user},{'text':1}))#fetch all the messages for the user
    user_text=''
    for text in user_texts:
        text = ast.literal_eval(text['text'])
        txt=''
        for t in text:#all the messages in an unic string
            txt += t
        user_text=user_text +' '+ txt
        users_messages[user]=user_text   
    
    
sentiments={}#k =user, value={text,score}
for k,v in users_messages.items():    
    sentiments[k]={'text':v,"score":sia.polarity_scores(v)}
sent1={}
for k,v in sentiments.items():#k =user, value=text
    sent1[k]=v['text']



In [ ]:
with open('sent1.txt', 'w') as outfile:
    json.dump(sent1, outfile)

In [17]:
from sklearn.feature_extraction.text import CountVectorizer
count_vectorizer = CountVectorizer()

sparse_matrix = count_vectorizer.fit_transform(sent1.values())

In [18]:
sparse_matrix
text_term_matrix = sparse_matrix.todense()
df = pd.DataFrame(text_term_matrix, 
                  columns=count_vectorizer.get_feature_names(), 
                  index=sent1.keys())
display(df)

,100,110,1793,1st,2000,2011,abandoned,ability,abolished,about,...,yet,yield,you,young,your,yourself,youth,youthful,zero,zone
5ead2b331f313a96eed3adb7,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5ead2b331f313a96eed3ba08,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5ead2b331f313a96eed3c552,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5ead2b331f313a96eed3c14f,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5ead2b331f313a96eed3c345,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5ead2b331f313a96eed3b0ac,0,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
5ead2b331f313a96eed3b781,0,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
5ead2b331f313a96eed3abc3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5ead2b331f313a96eed3c3a7,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [19]:
from sklearn.metrics.pairwise import cosine_similarity as distance

similarity_matrix = distance(df,df)
print(similarity_matrix)

[[1.         0.         0.         ... 0.22537447 0.         0.04578685]
 [0.         1.         0.31701473 ... 0.03194383 0.18898224 0.23362836]
 [0.         0.31701473 1.         ... 0.05155393 0.10166571 0.25136748]
 ...
 [0.22537447 0.03194383 0.05155393 ... 1.         0.05634362 0.18574538]
 [0.         0.18898224 0.10166571 ... 0.05634362 1.         0.13736056]
 [0.04578685 0.23362836 0.25136748 ... 0.18574538 0.13736056 1.        ]]


In [23]:
#compute the similarity matrix
sim_df = pd.DataFrame(similarity_matrix, columns=sent1.keys(), index=sent1.keys())
display(sim_df)


,5ead2b331f313a96eed3adb7,5ead2b331f313a96eed3ba08,5ead2b331f313a96eed3c552,5ead2b331f313a96eed3c14f,5ead2b331f313a96eed3c345,5ead2b331f313a96eed3ae9e,5ead2b331f313a96eed3af26,5ead2b331f313a96eed3c5c5,5ead2b331f313a96eed3ab85,5ead2b331f313a96eed3bbaf,...,5ead2b331f313a96eed3b92a,5ead2b331f313a96eed3ac33,5ead2b331f313a96eed3add8,5ead2b331f313a96eed3b840,5ead2b331f313a96eed3c36f,5ead2b331f313a96eed3b0ac,5ead2b331f313a96eed3b781,5ead2b331f313a96eed3abc3,5ead2b331f313a96eed3c3a7,5ead2b331f313a96eed3c00e
5ead2b331f313a96eed3adb7,1.000000,0.000000,0.000000,0.150970,0.000000,0.212000,0.061113,0.065512,0.056344,0.209657,...,0.145065,0.117851,0.294628,0.111111,0.090098,0.089087,0.089087,0.225374,0.000000,0.045787
5ead2b331f313a96eed3ba08,0.000000,1.000000,0.317015,0.128388,0.169031,0.180289,0.121268,0.111426,0.127775,0.138675,...,0.131590,0.066815,0.066815,0.000000,0.051081,0.050508,0.000000,0.031944,0.188982,0.233628
5ead2b331f313a96eed3c552,0.000000,0.317015,1.000000,0.189938,0.136399,0.210143,0.153775,0.229782,0.206216,0.159862,...,0.212373,0.080875,0.053916,0.000000,0.092744,0.040757,0.040757,0.051554,0.101666,0.251367
5ead2b331f313a96eed3c14f,0.150970,0.128388,0.189938,1.000000,0.227866,0.396070,0.384044,0.348636,0.229668,0.379824,...,0.453340,0.300240,0.300240,0.000000,0.336654,0.151307,0.211830,0.229668,0.113228,0.311060
5ead2b331f313a96eed3c345,0.000000,0.169031,0.136399,0.227866,1.000000,0.189618,0.245976,0.146490,0.188982,0.187523,...,0.194625,0.197642,0.079057,0.000000,0.135990,0.059761,0.000000,0.113389,0.074536,0.184289
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5ead2b331f313a96eed3b0ac,0.089087,0.050508,0.040757,0.151307,0.059761,0.283296,0.244998,0.087544,0.225877,0.280166,...,0.116311,0.283473,0.283473,0.000000,0.072239,1.000000,0.071429,0.000000,0.000000,0.000000
5ead2b331f313a96eed3b781,0.089087,0.000000,0.040757,0.211830,0.000000,0.169978,0.049000,0.000000,0.000000,0.168100,...,0.069786,0.047246,0.141737,0.000000,0.288957,0.071429,1.000000,0.000000,0.000000,0.146845
5ead2b331f313a96eed3abc3,0.225374,0.031944,0.051554,0.229668,0.113389,0.161255,0.170445,0.177177,0.114286,0.088596,...,0.250108,0.119523,0.119523,0.112687,0.137065,0.000000,0.000000,1.000000,0.056344,0.185745
5ead2b331f313a96eed3c3a7,0.000000,0.188982,0.101666,0.113228,0.074536,0.106000,0.183340,0.109187,0.112687,0.209657,...,0.116052,0.000000,0.058926,0.000000,0.067574,0.000000,0.000000,0.056344,1.000000,0.137361


In [37]:
def get3closest(sim_df,user):
    col=sim_df[user].sort_values(ascending=False)[1:]
    return list(col[0:3].index)

get3closest(sim_df,user)

[ObjectId('5ead2b331f313a96eed3b4f4'),
 ObjectId('5ead2b331f313a96eed3ab9f'),
 ObjectId('5ead2b331f313a96eed3b641')]

In [ ]:
from nltk.corpus import stopwords
nltk.download('stopwords')
set(stopwords.words('english'))